In [35]:
import cv2
import numpy as np

In [12]:
def on_mouse_click(event, x, y, flags, frame):
    if event == cv2.EVENT_LBUTTONUP:
        print(frame[y, x].tolist())

In [58]:
cap = cv2.VideoCapture("Squat1_8_9.avi")
# cap = cv2.VideoCapture('Squat2_16_17.avi')
# cap = cv2.VideoCapture('Squat3_11_9_10.avi')
# haveFrame, bg = cap.read()

i = 0
while cap.isOpened():
    haveFrame, im = cap.read()

    if i >= 30 or not haveFrame:
        bg = im
        break
    i += 1

while cap.isOpened():
    haveFrame, im = cap.read()
    if (not haveFrame) or (cv2.waitKey(10) & 0xFF == ord("q")):
        break

    diffc = cv2.absdiff(im, bg)
    diffg = cv2.cvtColor(diffc, cv2.COLOR_BGR2GRAY)
    bwmask = cv2.inRange(diffg, 35, 255)
    bwmask_median = cv2.medianBlur(bwmask, 11)

    kernel = np.ones((7, 7), np.uint8)
    bwmask_close = cv2.morphologyEx(bwmask_median, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(
        bwmask_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE
    )

    im_out_contour = im.copy()
    cv2.drawContours(im_out_contour, contours, -1, (0, 255, 0), 1)

    im_out_boundingbox = im.copy()
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if (
            w * h > diffg.shape[0] * diffg.shape[1] * 0.015
            and w * h < diffg.shape[0] * diffg.shape[1] * 0.2
            and x > 2
            and y > 2
            and x + w < diffg.shape[1] - 3
            and y + h < diffg.shape[0] - 3
            and h / w < 10
        ):
            cv2.rectangle(im_out_boundingbox, (x, y), (x + w, y + h), (0, 0, 255), 2)
            # all_boundingboxes.append([t, i , x, y, w, h])
            # i = i+1

    cv2.imshow("m", bwmask_close)
    cv2.imshow("contour", im_out_contour)
    cv2.imshow("bound box", im_out_boundingbox)

cap.release()
cv2.destroyAllWindows()